<a href="https://colab.research.google.com/github/Btsan/AffinityStyleTransfer/blob/main/AdaIN_Affinity_StyleTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook was made in google colab

datasets and the affinity estimator

In [ ]:
%%bash
pip install av
pip install --upgrade Pillow

git clone https://github.com/xiaolonw/UVC-1.git
cp -r UVC-1/* .

wget -q https://data.vision.ee.ethz.ch/csergi/share/davis/DAVIS-2017-Unsupervised-trainval-480p.zip
mkdir datasets
unzip -q DAVIS-2017-Unsupervised-trainval-480p.zip -d datasets

mkdir /root/.kaggle
cp kaggle.json /root/.kaggle/
chmod 600 ~/.kaggle/kaggle.json
kaggle competitions download -c painter-by-numbers -f train_1.zip
unzip -q train_1.zip -d datasets

mkdir saves
mkdir videos

  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0



ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
Cloning into 'UVC-1'...
ls: cannot access '/root/.kaggle/': No such file or directory
100%|##########| 4.76G/4.76G [00:58<00:00, 87.9MB/s]


vgg modified with one extra 1x1 convolutional layer and reflection padding

In [ ]:
%%bash
wget --load-cookies /tmp/cookies.txt \
"https://docs.google.com/uc?export=download&confirm=\
$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1zo2t62c26xaHhMBDyVcvjCHDNFOkalZs' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1zo2t62c26xaHhMBDyVcvjCHDNFOkalZs" \
-O pretrained.zip && rm -rf /tmp/cookies.txt
unzip -q pretrained.zip -d saves

dataset classes

In [ ]:
import torch
import torch.utils.data as data
from torchvision import transforms
from torchvision import io

from pathlib import Path

class ImageFolderDataset(data.Dataset):
	def __init__(self, folder, transform=None):
		super(ImageFolderDataset, self).__init__()
		self.folder = folder
		self.images = list(Path(self.folder).glob('**/*.*'))
		self.images.sort()
		self.transform = transform

	# i^th item from the dataset
	def __getitem__(self, index):
		index %= len(self.images)
		img = self.images[index]
		try:
			img = io.read_image(str(img))
		except Exception as e:
			# print("Corrupted: " + str(self.images[index]))
			self.images.pop(index)
			return self[index % len(self.images)]
		if self.transform is not None:
			img = self.transform(img)
		if img.shape[0] == 1:
			img = img.expand((3, -1, -1))
		elif img.shape[0] == 4:
			img =  img[:3, :, :]
		return img

	def __len__(self):
		return len(self.images)

	def name(self):
		return 'ImageFolderDataset'

class DavisDataset(data.Dataset):
    def __init__(self, img_folder, transform=None):
        super(DavisDataset, self).__init__()
        # root directory of folder
        self.folder = img_folder
        # search all files in subdirectories
        self.sub_folders = list(Path(self.folder).glob('*/'))
        self.frame_dict = {}
        for f in self.sub_folders:
            self.frame_dict[f] = list(Path(f).glob('**/*.*'))
            self.frame_dict[f].sort()
        self.lengths = [len(self.frame_dict[f]) - 20 for f in self.sub_folders]
        self.total_length = 0
        for l in self.lengths:
            self.total_length += l
        # transform function
        self.transform = transform
        self.batch_size=8
        
    # i^th item from the dataset
    def __getitem__(self, index):
        i = l = count = 0
        for i, l in enumerate(self.lengths):
            count += l
            if count >= index:
                break
        folder = self.sub_folders[i]
        sub_idx =  l - count + index - l # 30 - (120 - 100)
        frame1 = self.frame_dict[folder][sub_idx]
        frame2 = self.frame_dict[folder][sub_idx + 7] #1
        frame3 = self.frame_dict[folder][sub_idx + 2] #2
        frame4 = self.frame_dict[folder][sub_idx + 8] #3
        frame5 = self.frame_dict[folder][sub_idx + 3] #4
        frame6 = self.frame_dict[folder][sub_idx + 9]#10
        frame7 = self.frame_dict[folder][sub_idx + 4]#15
        frame8 = self.frame_dict[folder][sub_idx + 10]#20
        frame1 = io.read_image(str(frame1))
        frame2 = io.read_image(str(frame2))
        frame3 = io.read_image(str(frame3))
        frame4 = io.read_image(str(frame4))
        frame5 = io.read_image(str(frame5))
        frame6 = io.read_image(str(frame6))
        frame7 = io.read_image(str(frame7))
        frame8 = io.read_image(str(frame8))
        frames = torch.stack([frame1, frame2, frame3, frame4, frame5, frame6, frame7, frame8])
        if self.transform is not None:
            frames = self.transform(frames)
        return frames

    def __len__(self):
        return self.total_length

    def name(self):
        return 'DavisDataset'

AdaIN style classes

In [ ]:
from torch import nn
import torch.nn.functional as F

from torch import optim
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision.utils import save_image

from time import time

class VGGEncoder(nn.Module):
	def __init__(self, path=None):
		super(VGGEncoder, self).__init__()

		if path is not None:
			vgg = vgg_normalised = nn.Sequential( # Sequential,
				nn.Conv2d(3,3,(1, 1)),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(3,64,(3, 3)),
				nn.ReLU(), # 1 1
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(64,64,(3, 3)),
				nn.ReLU(),
				nn.MaxPool2d((2, 2),(2, 2),(0, 0),ceil_mode=True),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(64,128,(3, 3)),
				nn.ReLU(), # 2 1
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(128,128,(3, 3)),
				nn.ReLU(),
				nn.MaxPool2d((2, 2),(2, 2),(0, 0),ceil_mode=True),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(128,256,(3, 3)),
				nn.ReLU(), # 3 1
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(256,256,(3, 3)),
				nn.ReLU(),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(256,256,(3, 3)),
				nn.ReLU(),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(256,256,(3, 3)),
				nn.ReLU(),
				nn.MaxPool2d((2, 2),(2, 2),(0, 0),ceil_mode=True),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(256,512,(3, 3)),
				nn.ReLU(), # 4 1
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(512,512,(3, 3)),
				nn.ReLU(),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(512,512,(3, 3)),
				nn.ReLU(),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(512,512,(3, 3)),
				nn.ReLU(),
				nn.MaxPool2d((2, 2),(2, 2),(0, 0),ceil_mode=True),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(512,512,(3, 3)),
				nn.ReLU(),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(512,512,(3, 3)),
				nn.ReLU(),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(512,512,(3, 3)),
				nn.ReLU(),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(512,512,(3, 3)),
				nn.ReLU(),
			)
			save_path = Path(path)
			if save_path.is_file():
				try:
					state_dict = torch.load(str(save_path))
					vgg.load_state_dict(state_dict)
					print("Loaded vgg", save_path)
				except:
					print("Error: previous vgg failed to load")
					pass
			else:
				vgg = models.vgg19(pretrained=True).features.eval()
				print("Initializing new vgg")
		else:
			vgg = models.vgg19(pretrained=True).features.eval()
		features = list(vgg.children())
		#transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) ?
		self.enc_1 = nn.Sequential(*features[0:4]) 
		self.enc_2 = nn.Sequential(*features[4:11])
		self.enc_3 = nn.Sequential(*features[11:18])
		self.enc_4 = nn.Sequential(*features[18:31]) # content layer
		self.model = nn.Sequential(*features[0:30])

	def forward(self, x):
		x = self.model(x)
		self.content = F.relu(x).detach() # save content map for loss
		return x

	def compute_loss(self, g, s):
		g1 = self.enc_1(g)
		g2 = self.enc_2(g1)
		g3 = self.enc_3(g2)
		g4 = self.enc_4(g3)
		content_loss = F.mse_loss(g4, self.content)

		if s.dim() == 3:
			s = s.unsqueeze(0)

		s1 = self.enc_1(s)
		s2 = self.enc_2(s1)
		s3 = self.enc_3(s2)
		s4 = self.enc_4(s3)
		style_loss = F.mse_loss(g1.mean((2, 3)), s1.mean((2, 3))) \
			+ F.mse_loss(g2.mean((2, 3)), s2.mean((2, 3))) \
			+ F.mse_loss(g3.mean((2, 3)), s3.mean((2, 3))) \
			+ F.mse_loss(g4.mean((2, 3)), s4.mean((2, 3)))
		style_loss += F.mse_loss(g1.std((2, 3)), s1.std((2, 3))) \
			+ F.mse_loss(g2.std((2, 3)), s2.std((2, 3))) \
			+ F.mse_loss(g3.std((2, 3)), s3.std((2, 3))) \
			+ F.mse_loss(g4.std((2, 3)), s4.std((2, 3)))

		return content_loss, style_loss

class AdaIN(nn.Module):
	def __init__(self, encoder=None):
		super(AdaIN, self).__init__()

		if encoder is None:
			self.model = nn.Sequential(
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(512,256,(3, 3)),
				nn.ReLU(),
				nn.UpsamplingNearest2d(scale_factor=2),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(256,256,(3, 3)),
				nn.ReLU(),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(256,256,(3, 3)),
				nn.ReLU(),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(256,256,(3, 3)),
				nn.ReLU(),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(256,128,(3, 3)),
				nn.ReLU(),
				nn.UpsamplingNearest2d(scale_factor=2),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(128,128,(3, 3)),
				nn.ReLU(),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(128,64,(3, 3)),
				nn.ReLU(),
				nn.UpsamplingNearest2d(scale_factor=2),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(64,64,(3, 3)),
				nn.ReLU(),
				nn.ReflectionPad2d((1, 1, 1, 1)),
				nn.Conv2d(64,3,(3, 3)))
		else:
			children = [] 
			layers = list(encoder.children())
			for i in range(len(layers)-1, -1, -1):
				if isinstance(layers[i], nn.Conv2d):
					in_channels, out_channels = layers[i].weight.shape[:2]
					print("conv_{}".format(i), in_channels, out_channels)
					children.append(nn.Conv2d(in_channels, out_channels, 3, padding=1, padding_mode='reflect'))
					children.append(nn.ReLU())
				elif isinstance(layers[i], nn.MaxPool2d):
					children.append(nn.Upsample(scale_factor=2, mode='nearest'))
			children.pop()
			print(children)
			self.model = nn.Sequential(*children)

	def forward(self, content, style):
		target_std = style.std((2, 3))
		target_mean = style.mean((2, 3))
		adain = [F.instance_norm(content[i].unsqueeze(0), weight=target_std[i], bias=target_mean[i]) for i  in range(len(content))]
		adain = torch.cat(adain)
		adain = F.relu(adain) # assumes last vgg layer was conv2d
		y = self.model(adain)
		y = torch.tanh(y)
		return y

Affinity loss calculation class

In [ ]:
from scipy import signal

class AffinityLoss(nn.Module):
    def __init__(self, affinity_model):
        super(AffinityLoss, self).__init__()

        self.gray_encoder = affinity_model.gray_encoder
        self.nlm = affinity_model.nlm
        self.softmax = affinity_model.softmax

        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        self.mse = nn.MSELoss()

        for p in self.gray_encoder.parameters():
            p.requires_grad = False

    def compute_affinity(self, frame1, frame2):
        frame1 = self.normalize(frame1)
        frame2 = self.normalize(frame2)

        Fframe1 = self.gray_encoder(frame1)
        Fframe2 = self.gray_encoder(frame2)

        aff = self.nlm(Fframe1, Fframe2)
        return aff

    def forward(self, frame1, frame2, generated1, generated2, eye=0.):
        aff = self.compute_affinity(frame1, frame2)
        aff_norm_1 = self.softmax(aff * 10)
        aff = self.compute_affinity(generated1, generated2)
        aff_norm_2 = self.softmax(aff * 10)

        n, h, w = aff_norm_1.shape
        if eye > 0:
            diag = eye * torch.eye(h, w).unsqueeze(0).expand_as(aff_norm_1).to(aff_norm_1.device)
            aff_norm_1 += diag


        loss = F.mse_loss(aff_norm_1, aff_norm_2, reduction='none')
        gkern1d = torch.tensor(signal.gaussian(h, w), device=loss.device)
        gkern2d = torch.outer(gkern1d, gkern1d).unsqueeze(0).expand_as(loss)
        loss = (loss * gkern2d).mean()

        return loss

def total_variation_loss(x):
	N, C, H, W = x.shape
	a = torch.square(x[:, :,: H - 1, : W - 1] - x[:, :, 1:, : W - 1])
	b = torch.square(x[:, :, : H - 1, : W - 1] - x[:, :, : H - 1, 1:])
	return torch.sum(torch.pow(a + b, 1.25))

Load the affinity estimator

In [ ]:
from collections import OrderedDict
from libs.model import Model_switchGTfixdot_swCC_Res as Affinity

def get_affinity_loss(path="weights/checkpoint_latest.pth.tar"):
	aff_model = Affinity(pretrainRes=False, temp=1, uselayer=4)
	checkpoint = torch.load(path)

	new_state_dict = OrderedDict()
	for k, v in checkpoint["state_dict"].items():
		name = k[7:] # remove `module.`
		new_state_dict[name] = v

	aff_model.load_state_dict(new_state_dict)
	print("loaded affinity model")

	aff_model = AffinityLoss(aff_model)
	return aff_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
aff_loss = get_affinity_loss().to(device)

loaded affinity model


Training loop

In [ ]:
def test(encoder, decoder, name="test.mp4", imsize=256, video_path="motorbike", style_path="style.jpg"):
    # resize = transforms.Resize((imsize, imsize))
    dataset = ImageFolderDataset("datasets/DAVIS/JPEGImages/480p/" + video_path) #,resize
    dataloader = DataLoader(dataset, 4, shuffle=False)

    pred = []
    with torch.no_grad():
        style_im = io.read_image(style_path).unsqueeze(0).to(device) / 255
        style_ft = encoder(style_im).repeat(4, 1, 1, 1)
        del style_im
        for idx, im in enumerate(dataloader):
            im = im.to(device) / 255
            # im = im.unsqueeze(0).to(device) / 255
            p = decoder(encoder(im), style_ft)
            pred.append(p.cpu())

        if len(pred[0].shape) == 4:
                pred = torch.cat(pred, dim=0)
        else:
            pred = torch.stack(pred, dim=0)
        pred = pred.permute(0, 2, 3, 1)
        pred = (pred * 255 + 0.5).clamp(0, 255)
        io.write_video(name, pred, 15)
    return

def train(encoder, decoder, optimizer, content_dataset, style_dataset, affinity_loss, imsize=256, epochs=1, style_weight=1e-2, content_weight=1, affinity_weight=1, temporal_weight=1e-1, variation_weight=1e-6, affinity_eye=0):
    content_loader = DataLoader(content_dataset, 1, shuffle=True)
    style_loader = iter(DataLoader(style_dataset, content_dataset.batch_size, num_workers=1, sampler=data.RandomSampler(style_dataset, replacement=True, num_samples=epochs * content_dataset.batch_size * len(content_dataset))))

    for it in range(epochs):
        ts = tss = time()
        for idx, content_batch in enumerate(content_loader):
            style_batch = style_loader.__next__().to(device) / 255
            content_batch = content_batch.squeeze(0).to(device) / 255

            optimizer.zero_grad(True)
            pred = decoder(encoder(content_batch), encoder(style_batch))

            # print(content_batch.shape, pred.shape)
            aff_loss = affinity_loss(content_batch[:-1], content_batch[1:], pred[:-1], pred[1:], eye=affinity_eye)
            content_loss, style_loss = encoder.compute_loss(pred, style_batch)
            temporal_loss = torch.nn.functional.mse_loss(pred[1:], pred[:-1])
            variation_loss = total_variation_loss(pred)

            loss = style_weight * style_loss + content_weight * content_loss + affinity_weight * aff_loss + temporal_weight * temporal_loss
            loss.backward()
            optimizer.step()

            if idx % 250 == 0:
                print("{:>3}.{:<5} c_loss {:.4f}  s_loss {:.4f}  a_loss {:.4f}  t_loss {:.4f}  v_loss {:.4f} {:>10.3f}".format(
                    it, idx, content_loss.item(), style_loss.item(), aff_loss.item(), temporal_loss.item(), variation_loss.item(), time()-tss))
                tss = time()

        print("{} - {:.3f}s".format(it, time()-ts))
        test(encoder, decoder, "videos/test_{}.mp4".format(it))
        state_dict = decoder.model.state_dict()
        torch.save(state_dict, "saves/adain_decoder.pth")
        state_dict = optimizer.state_dict()
        torch.save(state_dict, "saves/adam.pth")
    return

initialize AdaIN encoder and decoder

In [ ]:
encoder = VGGEncoder("saves/vgg_normalised.pth").to(device)
decoder = AdaIN().to(device)
save_path = Path("saves/adain_decoder.pth")
if save_path.is_file():
    try:
        state_dict = torch.load(str(save_path))
        decoder.model.load_state_dict(state_dict)
        print("Loaded previous decoder", save_path)
    except:
        print("Error: previous decoder failed to load")
        pass
else:
    print("No previous save. Initializing new decoder")

optimizer = optim.Adam(decoder.parameters(), lr=1e-6, weight_decay=5e-5)
save_path = Path("saves/adam.pth")
if save_path.is_file():
    try:
        state_dict = torch.load(str(save_path))
        optimizer.load_state_dict(state_dict)
        print("Loaded previous optimizer", save_path)
    except:
        print("Error: previous optimizer failed to load")
        pass
else:
    print("No previous save. Initializing new optimizer")

resize = transforms.Resize((256, 256))
content_dataset = DavisDataset("datasets/DAVIS/JPEGImages/480p", resize)
style_dataset = ImageFolderDataset("datasets/train_1", resize)

call training

In [ ]:
train(encoder, decoder, optimizer, content_dataset, style_dataset, aff_loss, epochs=100, style_weight=3, affinity_weight=1e5, temporal_weight=10, variation_weight=0, affinity_eye=0.1)

torch.save(decoder.model.state_dict(), "saves/adain_decoder.pth")
torch.save(optimizer.state_dict(), "saves/adam.pth")